In [1]:
import polars as pl , glob , numpy as np,os

In [4]:
import polars as pl , glob , numpy as np,os
class BiddingLazyframe:
    def __init__(self, spec=-1, dataset_path=r'D:\allcode\hacks\Devcraft25\dataset'):
        self.type = 'bid'
        self.bid_columns = [
            'bidId','time_stamp','cookieId','user_agent','ip_addr','region','city','exchange',
            'domain','url','anonymousURL','adslotid','adslotwidth','adslotheight','adslotvisibility',
            'adslotformat','adslotfloorprice','creativeid','bidding_price','advertiser_id','user_profile_id'
        ]

        self.other_columns = [
            'bidId','time_stamp','log_type','cookieId','user_agent','ip_addr','region','city','exchange',
            'domain','url','anonymousURL','adslotid','adslotwidth','adslotheight','adslotvisibility',
            'adslotformat','adslotfloorprice','creativeid','bidding_price','paying_price','landing_url',
            'advertiser_id','final_user_profile_id'
        ]
        
        bid_dict_map = dict(map(lambda k: (f"column_{k[0]+1}",k[1]) , enumerate(self.bid_columns)))
        other_dict_map = dict(map(lambda k: (f"column_{k[0]+1}",k[1]) , enumerate(self.other_columns)))
        
        
        #define helper functions for preprocessing
        load_lf = lambda type , spec, dataset_path: pl.concat([pl.scan_csv(file, separator='\t', has_header=False) 
                                                               for file in glob.glob(f'{dataset_path}\\{type}.{'*' if spec == -1 else str(spec).zfill(2)}.txt')])
        get_bidId_set = lambda x: set(load_lf(x,spec,dataset_path).select('column_1').collect()['column_1'].to_list())
        add_col_from_set = lambda lf , type_set , col_name : lf.with_columns(pl.col('bidId').is_in(type_set).alias(col_name))
        get_rem_cols_lazy = lambda lf, col_names, imp_lf: lf.join(imp_lf.select(['bidId'] + col_names), on='bidId', how='left')
        
        
        
        self.df = load_lf('bid',spec,dataset_path).rename(bid_dict_map)
        self.impression , self.click , self.conversion = [get_bidId_set(attr) for attr in ['imp','clk','conv']]
        self.df = add_col_from_set(self.df,self.impression,'impression_bool')
        self.df = add_col_from_set(self.df,self.click,'click_bool')
        self.df = add_col_from_set(self.df,self.conversion,'conversion_bool')
        imp_lf = load_lf('imp', spec, dataset_path).rename(other_dict_map)
        self.df = get_rem_cols_lazy(self.df, ['paying_price', 'landing_url', 'final_user_profile_id'], imp_lf)
        self.df = self.df.with_columns([
                    (pl.col("time_stamp") // 1000000000).cast(pl.Utf8).str.strptime(pl.Date,'%Y%m%d').alias('date'),
                    (
                        ((pl.col("time_stamp") // 1000000 % 100).cast(pl.Int32) * 4) +
                        ((pl.col("time_stamp") // 10000 % 100).cast(pl.Int32) // 15)
                    ).alias("time_block"),
                    (pl.col("time_stamp") // 1000000000).cast(pl.Utf8).str.strptime(pl.Date, "%Y%m%d").dt.weekday().alias("day_of_week")
        ])
        
    def write_to_directory(self, output_dir, by_col="advertiser_id"):
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        unique_groups = self.df.select(by_col).unique().collect()[by_col].to_list()
        print(unique_groups)
        for group in unique_groups:
            print(f"Processing group: {group}")
            lf_subset = self.df.filter(pl.col(by_col) == group)
            lf_subset.sink_parquet(f"{output_dir}/{group}.parquet",compression='zstd',compression_level=12)
            print(f"Finished processing group: {group}")
        
        
lf = BiddingLazyframe()
pl.Config.set_tbl_rows(100)
data = lf.df.head(100).collect(streaming=True)
pl.Config.restore_defaults()
data

bidId,time_stamp,cookieId,user_agent,ip_addr,region,city,exchange,domain,url,anonymousURL,adslotid,adslotwidth,adslotheight,adslotvisibility,adslotformat,adslotfloorprice,creativeid,bidding_price,advertiser_id,user_profile_id,impression_bool,click_bool,conversion_bool,paying_price,landing_url,final_user_profile_id,date,time_block,day_of_week
str,i64,str,str,str,i64,i64,i64,str,str,str,str,i64,i64,i64,i64,i64,str,i64,i64,str,bool,bool,bool,i64,str,str,date,i32,i8
"""b382c1c156dcbbd5b9317cb50f6a74…",20130606000104008,"""Vh16OwT6OQNUXbj""","""mozilla/4.0 (compatible; msie …","""180.127.189.*""",80,87,1,"""tFKETuqyMo1mjMp45SqfNX""","""249b2c34247d400ef1cd3c6bfda4f1…",null,"""mm_11402872_1272384_3182279""",300,250,1,1,0,"""00fccc64a1ee2809348509b7ac2a97…",227,3427,"""null""",false,false,false,null,null,null,2013-06-06,0,4
"""7b6195de0d14203f92001da653bf1d…",20130606000104009,"""Vhkr1vpROHuhQWB""","""Mozilla/4.0 (compatible; MSIE …","""113.119.105.*""",216,217,2,"""trqRTuToMTNUjM9r5rMi""","""74419a072f8927222a1fd8aaa18cce…",null,"""433287550""",468,60,1,0,5,"""2f88fc9cf0141b5bbaf251cab07f4c…",300,3386,"""null""",true,false,false,18,"""361e128affece850342293213691a0…","""null""",2013-06-06,0,4
"""2ea9fe21cf7350fcb5696d8cff0bbe…",20130606000104012,"""VhKdLnuY3tlhXMa""","""mozilla/4.0 (compatible; msie …","""42.184.148.*""",65,69,1,"""trqRTvpogNlyDok4JKTI""","""134db65c2b66d8468d00bf42fd9f91…",null,"""mm_10032051_2374052_9219530""",950,90,1,1,0,"""23d6dade7ed21cea308205b3759400…",227,3427,"""null""",false,false,false,null,null,null,2013-06-06,0,4
"""8a15b98c8f9e60d4f92aaab01acf52…",20130606000104014,"""VhTVORqG36N6qMj""","""mozilla/5.0 (windows nt 5.1) a…","""114.100.37.*""",106,117,1,"""lsxSl559Xql7FmMs""","""8c9742e63497713b97ac7e780a8f9a…",null,"""mm_30232185_2681382_11190685""",950,90,0,1,0,"""23d6dade7ed21cea308205b3759400…",227,3427,"""null""",true,false,false,207,"""b2e35064f3549d447edbbdfb1f707c…","""10063,10684,10083,13403,10059,…",2013-06-06,0,4
"""faf17eac9cabf1be598f4e75f40d50…",20130606000104016,"""VhL01pk8OTkW3Mc""","""mozilla/5.0 (windows nt 5.1) a…","""58.100.240.*""",94,95,1,"""tK1NTu1YP5scFsf""","""e22930480589abcc1468854cb34033…",null,"""mm_10075660_3500949_11453278""",950,90,0,1,0,"""d5cecca9a6cbd7a0a48110f1306b26…",227,3358,"""null""",true,false,false,108,"""d29e59bf0f7f8243858b8183f14d44…","""10059,13866,10063,10111""",2013-06-06,0,4
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""5461589ac8f2672f413a5c5993c768…",20130606000104191,"""null""","""mozilla/4.0 (compatible; msie …","""113.111.121.*""",216,217,1,"""tFKETuqyMo1mjMp45SqfNX""","""249b2c34247d400ef1cd3c6bfda4f1…",null,"""mm_11402872_1272384_3182279""",300,250,1,1,0,"""d881a6c788e76c2c27ed1ef04f1195…",227,3358,"""null""",false,false,false,null,null,null,2013-06-06,0,4
"""42793ee70a154896e5ec829f2b15ee…",20130606000104193,"""Vhkr1uq-OtsPwdC""","""mozilla/4.0 (compatible; msie …","""118.182.244.*""",344,350,1,"""DFpETuxoGQdcFNKbuKz""","""ec29cecc6ec5e25077a01f13fde25e…",null,"""mm_10024662_3445902_11178345""",336,280,2,1,0,"""13606a7c541dcd9ca1948875a760bb…",227,3358,"""null""",true,false,false,14,"""d29e59bf0f7f8243858b8183f14d44…","""10006,10110""",2013-06-06,0,4
"""20fb85784972aebed3dda8d20ca87a…",20130606000104193,"""VhkSPnNDP8L8eYl""","""opera/9.80 (android; opera min…","""58.67.157.*""",216,217,1,"""trqRTuS8jZL7FmMs""","""1eb132233cd79996f8abdd03e2de66…",null,"""mm_40468498_3493773_11415894""",300,250,0,5,0,"""d881a6c788e76c2c27ed1ef04f1195…",227,3358,"""null""",true,false,false,89,"""d29e59bf0f7f8243858b8183f14d44…","""13866,10111""",2013-06-06,0,4


NameError: name 'pl' is not defined

In [ ]:
lf.write_to_directory('D:\\allcode\\hacks\\Devcraft25\\preprocessed')

In [ ]:
preprocessed_path = r"D:\allcode\hacks\Devcraft25\preprocessed"
import glob, os, polars as pl
from IPython.display import display
preprocessed_files = glob.glob(f"{preprocessed_path}/*.parquet")
comppressed_parquet_path = r"D:\allcode\hacks\Devcraft25\comp_preprocessed"

if not os.path.exists(comppressed_parquet_path): os.makedirs(comppressed_parquet_path)
for file in preprocessed_files:
    print(f"Loading {file}")
    df = pl.read_parquet(file)
    display(df.head(100_000))
    print(f"Finished loading {file}")